# Increasing Confidence Experiments Softmax

## Dependencies 

In [ ]:
import json

import numpy as np
import torch
import matplotlib.pyplot as plt

## Dependencies 

In [1]:
import json

import numpy as np
import torch
import matplotlib.pyplot as plt

# Increase Experts 

In [2]:
# === Softmax ===
n_classes = 10
random_expert_idx = 0
confs = []
exps = []
true = []
path = "softmax_increase_confidence/"
n_experts = 4
p_experts = [0.2, 0.4, 0.6, 0.8, 0.95]
for p in p_experts:
    model_name = '_' + str(p) + '_confidence'
    with open(path + 'confidence_multiple_experts' + model_name + '.txt', 'r') as f:
        conf = json.loads(json.load(f))
    with open(path + 'expert_predictions_multiple_experts' + model_name + '.txt', 'r') as f:
        exp_pred = json.loads(json.load(f))
    with open(path + 'true_label_multiple_experts' + model_name + '.txt', 'r') as f:
        true_label = json.loads(json.load(f))
    true.append(true_label['test'])
    exps.append(exp_pred['test'])
    c = torch.tensor(conf['test'])
    print(c.shape)
    # DANI Correction ===
    c = c.softmax(dim=1)
    # DANI Correction ===

    temp = 0
    for i in range(n_experts):
        temp += c[:, (n_classes + n_experts) - (i + 1)]
    prob = c / (1.0 - temp).unsqueeze(-1)
    confs.append(prob)


torch.Size([9216, 11])
torch.Size([9216, 12])
torch.Size([9216, 14])
torch.Size([9216, 16])
torch.Size([9216, 18])


In [3]:
confs[2][0]

tensor([9.2400e-06, 5.0780e-05, 3.9151e-05, 9.9978e-01, 2.9459e-05, 1.6903e-04,
        9.1648e-07, 3.2866e-06, 3.7994e-07, 9.5552e-07, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00])

# Increase confidence 

In [4]:
# === Softmax ===
n_classes = 10
random_expert_idx = 0
confs = []
exps = []
true = []
path = "softmax_increase_confidence/"
n_experts = 4
p_experts = [0.2, 0.4, 0.6, 0.8, 0.95]
for p in p_experts:
    model_name = '_' + str(p) + '_confidence'
    with open(path + 'confidence_multiple_experts' + model_name + '.txt', 'r') as f:
        conf = json.loads(json.load(f))
    with open(path + 'expert_predictions_multiple_experts' + model_name + '.txt', 'r') as f:
        exp_pred = json.loads(json.load(f))
    with open(path + 'true_label_multiple_experts' + model_name + '.txt', 'r') as f:
        true_label = json.loads(json.load(f))
    true.append(true_label['test'])
    exps.append(exp_pred['test'])
    c = torch.tensor(conf['test'])
    print(c.shape)
    # DANI Correction ===
    c = c.softmax(dim=1)
    # DANI Correction ===

    temp = 0
    for i in range(n_experts):
        temp += c[:, (n_classes + n_experts) - (i + 1)]
    prob = c / (1.0 - temp).unsqueeze(-1)
    confs.append(prob)


# Conformal

In [5]:
import numpy as np
import torch

# Quantile Model calculations ===
def qhat_conformal_model(model, x_calib, y_calib, alpha=0.9):
    n = x_calib.shape[0]
    # Compute softmax for K classes, without deferral.
    out_logits = model(x_calib.float(), logits_out=True)
    # Discard deferal category
    out = out_logits[:, :-1].softmax(dim=1)

    sorted, pi = out.sort(dim=1, descending=True)
    scores = sorted.cumsum(dim=1).gather(1, pi.argsort(1))[range(y_calib.shape[0]), y_calib]
    # Get the score quantile
    qhat = torch.quantile(scores, np.ceil((n + 1) * (1 - alpha)) / n)
    return qhat


def qhat_conformal_model_logits(out_logits, y_calib, alpha=0.9):
    n = len(y_calib)
    # Discard deferal category
    out = out_logits[:, :-1].softmax(dim=1)

    # Scores
    sorted, pi = out.sort(dim=1, descending=True)
    # Add until y_true (y_calib)
    scores = sorted.cumsum(dim=1).gather(1, pi.argsort(1))[range(y_calib.shape[0]), y_calib]

    # Get the score quantile
    qhat = torch.quantile(scores, np.ceil((n + 1) * (1 - alpha)) / n)
    return qhat


# Quantile Expert calculations ===
def qhat_conformal_expert(model, expert_fn, x_calib, y_calib, device, alpha=0.9, statistic=1):
    n = x_calib.shape[0]
    out = model(x_calib.float())

    sorted, pi = out.sort(dim=1, descending=True)
    defer_dim = out.shape[-1] - 1  # defer dim number

    # Expert correctnes ===
    m = expert_fn(x_calib, y_calib).to(device)
    expert_correct = (m == y_calib)
    # Obtain "stop" condition depending on statistic
    stop_label = torch.zeros(y_calib.shape, dtype=torch.int64).to(device)

    # Statistic 1 ===
    if statistic == 1:
        # expert correct, pi_j = K+1
        stop_label[expert_correct] = defer_dim

        # expert incorrect, pi_j = argmin(y*, K+1) out
        out_truelabel = out[range(y_calib.shape[0]), y_calib]
        out_deferal = out[:, -1]
        true_gt_deferal = out_truelabel > out_deferal
        stop_label[(~expert_correct) & true_gt_deferal] = defer_dim  # true > deferal: set deferal
        stop_label[(~expert_correct) & (~true_gt_deferal)] = y_calib[(~expert_correct) & (~true_gt_deferal)]  # deferal > true: set true

    # Statistic 2 ===
    elif statistic == 2:
        stop_label = defer_dim  # pi_j = K+1

    # Statistic 3 ===
    elif statistic == 3:  # TODO: NOT USE
        stop_label = defer_dim  # pi_j = K+1

        # condition = (expert_correct) & (out[:, -1] > out[range(y_calib.shape[0]), y_calib])
        # stop_label[condition] = defer_dim  # pi_j = K+1
        # stop_label[~condition] = y_calib[~condition]  # pi_j = K+1

    # Get Scores
    scores = sorted.cumsum(dim=1).gather(1, pi.argsort(1))[range(y_calib.shape[0]), stop_label]  # stop label!
    # Get the score quantile
    qhat = torch.quantile(scores, np.ceil((n + 1) * (1 - alpha)) / n)
    return qhat


def qhat_conformal_expert_logits(out_logits, exp_pred, y_calib, device, alpha=0.9, statistic=1):
    n = len(y_calib)
    out = out_logits.softmax(dim=1)

    sorted, pi = out.sort(dim=1, descending=True)
    defer_dim = out.shape[-1] - 1  # defer dim number

    # Expert correctness ===
    m = exp_pred.to(device)
    expert_correct = (m == y_calib)
    # Obtain "stop" condition depending on statistic
    stop_label = torch.zeros(y_calib.shape, dtype=torch.int64).to(device)

    # Statistic 1 ===
    if statistic == 1:
        # expert correct, pi_j = K+1
        stop_label[expert_correct] = defer_dim

        # expert incorrect, pi_j = argmin(y*, K+1) out
        out_truelabel = out[range(y_calib.shape[0]), y_calib]
        out_deferal = out[:, -1]
        true_gt_deferal = out_truelabel > out_deferal
        stop_label[(~expert_correct) & true_gt_deferal] = defer_dim  # true > deferal: set deferal
        stop_label[(~expert_correct) & (~true_gt_deferal)] = y_calib[(~expert_correct) & (~true_gt_deferal)]  # deferal > true: set true

    # Statistic 2 ===
    elif statistic == 2:
        stop_label = defer_dim  # pi_j = K+1

    # Statistic 3 ===
    elif statistic == 3:  # TODO: NOT USE
        stop_label = defer_dim  # pi_j = K+1

        # condition = (expert_correct) & (out[:, -1] > out[range(y_calib.shape[0]), y_calib])
        # stop_label[condition] = defer_dim  # pi_j = K+1
        # stop_label[~condition] = y_calib[~condition]  # pi_j = K+1

    # Get Scores
    scores = sorted.cumsum(dim=1).gather(1, pi.argsort(1))[range(y_calib.shape[0]), stop_label]  # stop label!
    # Get the score quantile
    qhat = torch.quantile(scores, np.ceil((n + 1) * (1 - alpha)) / n)
    return qhat


# Prediction Sets Model ===
def get_prediction_sets_model(model, x_test, q_hat):
    out_logits = model(x_test.float(), logits_out=True)
    # Discard defedral category
    out = out_logits[:, :-1].softmax(dim=1)

    test_sorted, test_pi = out.sort(dim=1, descending=True)
    sizes = (test_sorted.cumsum(dim=1) > q_hat).int().argmax(dim=1)
    prediction_sets = [test_pi[i][:(sizes[i] + 1)] for i in range(sizes.shape[0])]
    return prediction_sets


def get_prediction_sets_model_logits(out_logits, q_hat):
    # Discard defedral category
    out = out_logits[:, :-1].softmax(dim=1)

    test_sorted, test_pi = out.sort(dim=1, descending=True)
    sizes = (test_sorted.cumsum(dim=1) > q_hat).int().argmax(dim=1)
    prediction_sets = [test_pi[i][:(sizes[i] + 1)] for i in range(sizes.shape[0])]
    return prediction_sets


# Prediction Sets Expert ===
def get_prediction_sets_expert(model, x_test, q_hat, statistic=2):
    # Deploy (output=list of lenght n, each element is tensor of classes)
    out = model(x_test.float())
    defer_dim = out.shape[-1] - 1  # defer dim number

    test_sorted, test_pi = out.sort(dim=1, descending=True)
    sizes = (test_sorted.cumsum(dim=1) > q_hat).int().argmax(dim=1)
    if statistic == 2:
        stop_at_kplus1 = [torch.where(test_pi[i] == defer_dim)[0] for i in range(out.shape[0])]
        prediction_sets = [test_pi[i][:(idx + 1)] for i, idx in enumerate(stop_at_kplus1)]
    else:
        prediction_sets = [test_pi[i][:(sizes[i] + 1)] for i in range(sizes.shape[0])]

    # Drop deferral dimension (K+1)
    prediction_sets = [set_i[set_i != defer_dim] for set_i in prediction_sets]
    return prediction_sets


def get_prediction_sets_expert_logits(out_logits, q_hat, statistic=2):
    defer_dim = out_logits.shape[-1] - 1  # defer dim number
    out = out_logits.softmax(dim=1)

    test_sorted, test_pi = out.sort(dim=1, descending=True)
    if statistic == 2:
        stop_at_kplus1 = [torch.where(test_pi[i] == defer_dim)[0] for i in range(out.shape[0])]
        prediction_sets = [test_pi[i][:(idx + 1)] for i, idx in enumerate(stop_at_kplus1)]
    else:
        sizes = (test_sorted.cumsum(dim=1) > q_hat).int().argmax(dim=1)
        prediction_sets = [test_pi[i][:(sizes[i] + 1)] for i in range(sizes.shape[0])]

    # Drop deferral dimension (K+1)
    prediction_sets = [set_i[set_i != defer_dim] for set_i in prediction_sets]
    return prediction_sets


# Cardinality ===
def set_cardinality(pred_set):
    """
    Calculate cardinality of different sets.
    """
    cardinality = [torch.tensor(set_i.shape) for set_i in pred_set]
    return torch.tensor(cardinality)


# Defer t ===
def defer_to_expert(model_cardinality, expert_cardinality, device):
    r"""
    Calculate defer to expert (rejection) depending on model and expert cardinalities.

    Args:
        model_cardinality:
        expert_cardinality:
        get:

    Returns:
        reject:
    """
    # 1 = Defer
    reject = torch.tensor(
        [model_card_i > expert_card_i for (model_card_i, expert_card_i) in
         zip(model_cardinality, expert_cardinality)]
    )
    # If it is equal -> Standard L2D
    reject_tiebreak = torch.tensor(
        [model_card_i == expert_card_i for (model_card_i, expert_card_i) in
         zip(model_cardinality, expert_cardinality)]
    )
    return reject.to(device), reject_tiebreak.to(device)


In [6]:
# 1 expert rando, 3 with prob 0.8 correct
probs = confs[-2]
experts = exps[-2]
y_true = true[-2]

In [7]:
n_val = int(0.8 * len(y_true))
n_test = len(y_true) - n_val

# Validation

In [8]:
## 2. get Q_hat
probs_val = probs[:n_val, 10:]
experts_val = [exp[:n_val] for exp in experts]
y_true_val = y_true[:n_val]

_, predicted = torch.max(probs[:n_val].data, 1)
r = (predicted >= n_classes - n_experts)

# === Only on deferred samples
probs_val = probs_val[r]
experts_val = [np.array(exp)[r] for exp in experts_val]
y_true_val = np.array(y_true_val)[r]

# 2.b Sort J model outputs for experts
probs_experts = probs[:n_val, 10:]
probs_experts = probs_experts[r]
sort, pi = probs_experts.sort(dim=1, descending=True)


# Check if experts are correct 
correct_exp = (np.array(experts_val) == np.array(y_true_val)).T
# Swap order to match confidence ordering
correct_exp = np.flip(correct_exp).copy()  # copy needed!

# idx for correct experts: [[0,1,2], [1,2], [], ...]
correct_exp_idx = [np.where(correct_exp_i)[0] for correct_exp_i in correct_exp]

# obtain the last expert to be retrieved. If empty, then add all values.
# indexes are not the real expert index, but the sorted indexes, e.g. [[1, 0 ,2],  [1,0], [], ...]
pi_corr_exp = [probs_experts[i, corr_exp].sort(descending=True)[1] for i, corr_exp in enumerate(correct_exp)]
pi_corr_exp_stop = [pi_corr_exp_i[-1] if len(pi_corr_exp_i)!=0 else -1 for pi_corr_exp_i in pi_corr_exp]  # last expert

# obtain real expert index back, e.g. [2,1,-1,...]
pi_stop = [correct_exp_idx[i][pi_corr_exp_stop_i] if len(correct_exp_idx[i])!=0 else -1 for i, pi_corr_exp_stop_i in enumerate(pi_corr_exp_stop)]


# =========
n_val = n_val
alpha = 0.95
scores = sort.cumsum(dim=1).gather(1, pi.argsort(1))[range(len(torch.tensor(pi_stop))), torch.tensor(pi_stop)]
qhat = torch.quantile(scores, np.ceil((n + 1) * (1 - alpha)) / n)

qhat

tensor(0.5678)

In [9]:
qhat = torch.quantile(scores, 0.05)
qhat

tensor(0.3136)

In [10]:
scores

tensor([0.9273, 0.8635, 0.8593,  ..., 1.6457, 1.6705, 0.5518])

In [11]:
pi

tensor([[2, 1, 0, 3],
        [2, 1, 0, 3],
        [0, 1, 2, 3],
        ...,
        [1, 0, 2, 3],
        [1, 0, 2, 3],
        [0, 1, 2, 3]])

# Test

In [445]:
n_classes = 10
n_experts = 4

In [405]:
probs_test = probs[n_val:, 10:]
experts_test = [exp[n_val:] for exp in experts]
y_true_test = y_true[n_val:]

# 2.b Sort J model outputs for experts
probs_experts = probs[n_val:, 10:]
sort_test, pi_test = probs_experts.sort(dim=1, descending=True)


sizes = (sort_test.cumsum(dim=1) > qhat).int().argmax(dim=1)
prediction_sets = [pi_test[i][:(sizes[i] + 1)] for i in range(sizes.shape[0])]  # not allow empty sets


In [417]:
_, predicted = torch.max(probs[n_val:].data, 1)
r = (predicted >= n_classes - n_experts)

In [423]:
r = (predicted >= n_classes - n_experts)

In [416]:
probs_test

tensor([[0.8161, 0.8437, 0.7678, 0.1060],
        [0.8036, 0.7980, 0.8149, 0.1306],
        [0.8327, 0.8606, 0.7507, 0.0810],
        ...,
        [0.8224, 0.8567, 0.8424, 0.0901],
        [0.2448, 0.2486, 0.2371, 0.0964],
        [0.7957, 0.8283, 0.7771, 0.1163]])

In [404]:
correct_exp_idx[:10]

[array([0, 1, 2]),
 array([1, 2]),
 array([0]),
 array([], dtype=int64),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([0, 2]),
 array([0, 1, 2]),
 array([1, 2]),
 array([], dtype=int64)]

In [403]:

# Check if experts are correct 
correct_exp = (np.array(experts_test) == np.array(y_true_test)).T
# Swap order to match confidence ordering
correct_exp = np.flip(correct_exp).copy()  # copy needed!

# idx for correct experts: [[0,1,2], [1,2], [], ...]
correct_exp_idx = [np.where(correct_exp_i)[0] for correct_exp_i in correct_exp]


In [472]:
correct_exp_idx

[array([0]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([1, 2]),
 array([], dtype=int64),
 array([3]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([0, 1, 2]),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([0]),
 array([1]),
 array([1]),
 array([1, 2]),
 array([0]),
 array([0, 2]),
 array([1]),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([0, 2]),
 array([0, 2]),
 array([0]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([0, 2]),
 array([], dtype=int64),
 array([1]),
 array([2, 3]),
 array([2]),
 array([], dtype=int64),
 array([0, 2]),
 array([], dtype=int64),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([3]),
 array([0, 3]),
 array([0, 1, 2]),
 array([0, 1]),
 array([0, 1, 2]),
 array([], dtype=int64),
 array([2]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([1]),
 array([0, 1, 2]),
 array([0]),
 array([], dtype=int64),